# Table of Contents
- [Imports](#Import)
- [Data Read In](#Data-Read-In)
- [Hierarchical GLM](#Hierarchical-GLM)

# Imports
[back to top](#Table-of-Contents)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
from bambi import Model, Prior
import pymc3 as pm

# Data Read In
[back to top](#Table-of-Contents)

In [4]:
root = "../data/"
survNoNA = pd.read_csv(root+"survCleanWithSameCols.csv", index_col=0).sort_values(["year", "sitecode"])
survSomeNA = pd.read_csv(root+"survCleanWithSomeNA.csv", index_col=0).sort_values(["year", "sitecode"])
survNoNA.head()

,carRiskScore,sitecode,year,age,sex,grade,race4,race7,stheight,stweight,bmi,qnobese,qnowt,q13,q18,q25,q26,q47,q57
122833,78.947368,CH,2005,1.0,2.0,1.0,2.0,3.0,1.80,72.58,22.401235,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
122834,0.000000,CH,2005,3.0,2.0,1.0,2.0,3.0,1.96,113.40,29.518950,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
122836,105.263158,CH,2005,3.0,2.0,1.0,2.0,3.0,1.78,60.33,19.041156,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0
122839,157.894737,CH,2005,3.0,2.0,1.0,2.0,3.0,1.78,61.69,19.470395,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0
122840,0.000000,CH,2005,3.0,2.0,1.0,4.0,7.0,1.78,69.85,22.045828,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0


In [5]:
survSomeNA.head()

,age,bmi,grade,q13,q18,q25,q26,q47,q57,qnobese,...,q32,q69,q46,q19,q12,q8,q9,q41,q40,q27
122831,6.0,NaN,3.0,1.0,1.0,2.0,2.0,1.0,2.0,NaN,...,1.0,1.0,1.0,NaN,1.0,3.0,1.0,1.0,1.0,2.0
122832,NaN,18.048443,1.0,1.0,1.0,2.0,2.0,6.0,2.0,NaN,...,3.0,1.0,4.0,2.0,5.0,4.0,4.0,2.0,5.0,2.0
122833,1.0,22.401235,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,...,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0,3.0,2.0
122834,3.0,29.518950,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,7.0,1.0,2.0,3.0,5.0,1.0,1.0,4.0,1.0
122835,3.0,21.520933,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,...,1.0,1.0,1.0,2.0,1.0,5.0,1.0,1.0,1.0,2.0


# Hierarchical GLM
[back to top](#Table-of-Contents)

In [13]:
year_idxs, years = pd.factorize(survNoNA["year"]) # needed for hierarchical model
county_idxs, counties = pd.factorize(survNoNA["sitecode"]) # needed for hierarchical model
coords = {
    "county": counties,
    "obs_county_id": np.arange(len(county_idxs)), # check these, seems like it just give 0 to len of array
    "year" : years,
    "obs_year_id" : np.arange(len(year_idxs))
}

In [ ]:
with pm.Model(coords=coords) as model:  # model specifications in PyMC3 are wrapped in a with-statement
    county_idx = pm.Data("county_idx", county_idxs, dims="obs_id")
     
    # Setting up data
    X = dat.drop(["carRiskScore", "sitecode", "year"], axis=1)
    X.insert(0, "intercept", 0)
    y = dat.carRiskScore
    
    # setting up betas
    qBetas = {}
    for i, q in enumerate(X.columns):
        # setting hyperparameters
        yearMu, yearVar = 0, .001
    
        # Define priors for highest hierarchy: year
        yearScales = pm.Normal("yearScale"+str(i), yearMu, sigma=yearVar)
        yearShapes = pm.Gamma("yearShape"+str(i), yearMu, sigma=yearVar)
        yearScalesVar = .01#pm.Normal("yearScaleVar", , sigma=) # fixed or random?
        yearShapesVar = .01#pm.Gamma("yearShapeVar", , sigma=) # fixed or random?
        
        # Define priors for lowest hierarchy: county
        countyScales = pm.Normal("countyScale"+str(i), yearScales, sigma=yearScalesVar, dims="year")
        countyShapes = pm.Normal("countyShape"+str(i), yearShapes, sigma=yearShapesVar, dims="year")
        
        # Defining betas
        qBeta = pm.Normal("beta"+str(i), countyScales, sigma=countyShapes, dims="county")
        qBetas["beta"+str(i)] = {"coef" : q_beta, "countyScale" : countyScales, 
                                 "countyShape" : countyShapes, "yearScale" : yearScales,
                                 "yearShapes" : yearShapes}
    
    # getting scale for y based on betas
    mu_est = 0
    for i, b in enumerate(qBetas.keys):
        mu_est += qBetas[b]["coef"][countyIdx][yearIdx] * X.iloc[:,i] # check how to do year and county idx
        
    # setting shape for y
    # Define priors for highest hierarchy: year
    yearScales = pm.Normal("yearScale", yearMu, sigma=yearVar)
    yearShapes = pm.Gamma("yearShape", yearMu, sigma=yearVar)
    yearScalesVar = .01#pm.Normal("yearScaleVar", , sigma=) # fixed or random?
    yearShapesVar = .01#pm.Gamma("yearShapeVar", , sigma=) # fixed or random?
    # Define priors for lowest hierarchy: county
    countyScales = pm.Normal("countyScale", yearScales, sigma=yearScalesVar, dims="year")
    countyShapes = pm.Normal("countyShape", yearShapes, sigma=yearShapesVar, dims="year")
    yShape = pm.Gamma("yShape", countyScales, sigma=countyShapes, dims="county")

    # Define likelihood
    likelihood = pm.Gamma(
        "y",
        alpha=np.exp(mu_est), # vs just mu_est?
        beta=yShape, # vs fixed?
        observed=dat["car_risk_score"],
        dims="obs_id") # check